
### Casper's Ghost Kitchen Destroyer

Select `Run All` to destroy Casper's Databricks environment

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
%restart_python

In [ ]:
import sys
sys.path.append('./utils')

CATALOG = dbutils.widgets.get("CATALOG")
print(f"Destroying resources for catalog: {CATALOG}")

state = None
try:
    from uc_state import create_state_manager
    state = create_state_manager(CATALOG)
    print(f"State manager initialized for {CATALOG}")
except Exception as e:
    print(f"⚠️ Could not initialize state manager: {e}")
    print("This usually means the catalog or state table doesn't exist. Skipping resource cleanup.")

In [ ]:
if state:
    try:
        results = state.clear_all()
        for rtype, info in results.items():
            if info["successful"]:
                print(f"✅ {rtype}: deleted {len(info['successful'])} resources")
            if info["failed"]:
                print(f"⚠️ {rtype}: {len(info['failed'])} failures")
                for f in info["failed"]:
                    print(f"   - {f['name']}: {f['reason']}")
    except Exception as e:
        print(f"⚠️ clear_all() failed: {e}")
        print("Continuing with catalog drop...")
else:
    print("No state manager available. Skipping resource cleanup, proceeding to catalog drop.")

In [ ]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

agent_names = [
    f"{CATALOG}-menu-knowledge",
    f"{CATALOG}-inspection-knowledge",
    f"{CATALOG}-menu-supervisor",
]

print(f"Cleaning up agents for catalog: {CATALOG}")

# 1. Look up tile IDs from /api/2.0/tiles, then delete via /api/2.0/tiles/{tile_id}
tile_ids_for_agents = {}
try:
    resp = w.api_client.do("GET", "/api/2.0/tiles")
    print(resp)
    tiles = resp.get("tiles", [])
    for tile in tiles:
        tile_name = tile.get("name", "")
        tile_id = tile.get("tile_id", "")
        if tile_name in agent_names:
            tile_ids_for_agents[tile_name] = tile_id
            print(f"   Found tile: name={tile_name}, id={tile_id}")
except Exception as e:
    print(f"⚠️ Could not list tiles: {e}")

# 2. Delete agents via tiles API -- try tile_id first, then agent name
for name in agent_names:
    deleted = False
    print(f"Deleting agent {name}")
    tile_id = tile_ids_for_agents.get(name)

    # Try DELETE with tile_id
    if tile_id:
        try:
            w.api_client.do("DELETE", f"/api/2.0/tiles/{tile_id}")
            print(f"✅ Deleted agent {name} via tiles API (id={tile_id})")
            deleted = True
        except Exception as e:
            print(f"   DELETE /api/2.0/tiles/{tile_id} failed: {e}")

    # Try DELETE with agent name
    if not deleted:
        try:
            w.api_client.do("DELETE", f"/api/2.0/tiles/{name}")
            print(f"✅ Deleted agent {name} via tiles API (name)")
            deleted = True
        except Exception as e:
            print(f"   DELETE /api/2.0/tiles/{name} failed: {e}")

    if not deleted:
        print(f"⚠️ {name}: could not delete via tiles API")

# 3. Delete serving endpoints that belong to this catalog
try:
    all_eps = list(w.serving_endpoints.list())
    for ep in all_eps:
        if ep.name and CATALOG in ep.name:
            try:
                w.serving_endpoints.delete(name=ep.name)
                print(f"✅ Deleted serving endpoint: {ep.name}")
            except Exception as e:
                print(f"   Could not delete {ep.name}: {e}")
except Exception as e:
    print(f"⚠️ Could not list endpoints: {e}")

In [0]:
%sql
DROP CATALOG IF EXISTS IDENTIFIER(:CATALOG) CASCADE;